### Membuat Peta Interaktif Dasar dengan Folium dan GeoPandas

In [1]:
!pip install folium geopandas

Reshimming asdf python...


In [1]:
import geopandas as gpd
from shapely.geometry import box

# Membaca data wilayah administratif
gdf = gpd.read_file("./data/BATAS KABUPATEN KOTA DESEMBER 2019 DUKCAPIL/")
gdf['area'] = gdf.geometry.area

# Memberi batasan supaya map tidak berat nantinya
minx, miny, maxx, maxy = 105.0, -2.5, 110.0, 0.0  # Definisi bounding box
# Membuat poligon bounding box
bbox = box(minx, miny, maxx, maxy)

# Seleksi objek yang berada dalam bounding box
gdf = gdf[gdf.geometry.within(bbox)]
gdf.head()

# Memeriksa data
gdf.head()

/var/folders/0c/s4bn1y5d3k3243ym_hrjnyqh0000gp/T/ipykernel_17841/3125067331.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf.geometry.area


,KAB_KOTA,geometry,area
23,BANGKA,"MULTIPOLYGON (((105.81694 -1.80492, 105.81438 ...",0.245180
24,BANGKA BARAT,"POLYGON ((105.59667 -1.52692, 105.59736 -1.528...",0.234482
231,KOTA PANGKALPINANG,"POLYGON ((106.13595 -2.06287, 106.13702 -2.064...",0.008503


In [2]:
import folium

# Menentukan pusat peta berdasarkan titik tengah data
centroid = gdf.geometry.centroid.iloc[0]
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=10)

# Menampilkan peta
m

/var/folders/0c/s4bn1y5d3k3243ym_hrjnyqh0000gp/T/ipykernel_17841/2180587306.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid.iloc[0]


In [3]:
# Menambahkan GeoDataFrame ke peta
folium.GeoJson(gdf).add_to(m)

# Menampilkan peta
m

In [4]:
# Menambahkan marker untuk setiap kota
for idx, row in gdf.iterrows():
    folium.Marker(
        location=[row.geometry.centroid.y, row.geometry.centroid.x],
        popup=row['KAB_KOTA'],  # Menggunakan nama kota sebagai popup
        tooltip=f"NAMA: {row['KAB_KOTA']} || AREA: {row['area']}"
    ).add_to(m)

# Menampilkan peta
m

In [5]:
# Menambahkan GeoDataFrame dengan popup dan tooltip
folium.GeoJson(
    gdf,
    name="Wilayah Kota",
    tooltip=folium.GeoJsonTooltip(fields=["KAB_KOTA", "area"],
                                  aliases=["Nama Kota:", "Area:"]),
    popup=folium.GeoJsonPopup(fields=["KAB_KOTA", "area"],
                              aliases=["Nama Kota:", "Area:"])
).add_to(m)

# Menampilkan peta
m

In [6]:
# Menambahkan kontrol layer
folium.LayerControl().add_to(m)

# Menampilkan peta
m

In [7]:
# Menambahkan GeoDataFrame dengan gaya khusus
folium.GeoJson(
    gdf,
    name="Wilayah Kota",
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.5
    }
).add_to(m)

# Menampilkan peta
m

In [8]:
# Membuat peta choropleth berdasarkan populasi
folium.Choropleth(
    geo_data=gdf,
    data=gdf,
    columns=['KAB_KOTA', 'area'],
    key_on='feature.properties.KAB_KOTA',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Area'
).add_to(m)

# Menampilkan peta
m

In [9]:
# Menyimpan peta sebagai file HTML
m.save("./output/peta_interaktif.html")